In [5]:
import urllib.parse
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary
# 日経のプレスリリースを確認する
# まずは、日経のプレスリリースのページをブラウザで確認
# https://www.nikkei.com/pressrelease/
#　日経のプレスリリースを検索
# ブラウザーを起動
options = Options()
# 必要ならproxyサーバを設定する
#PROXY = '127.0.0.1:8080'
#options.add_argument('--proxy-server=http://%s' % PROXY)
# ブラウザはヘッドレス（表示しない）とする
options.add_argument('--headless')
# Chromeのドライバを得る
browser = webdriver.Chrome(options=options)

# 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
browser.implicitly_wait(3)

# 検索キーワードをURLエンコードする
keywords = urllib.parse.quote('パナソニック　AND 発表日：2021')
url = 'https://www.nikkei.com/pressrelease/?searchKeyword='+ keywords + '&au=0'

# URLを読み込む
browser.get(url)
# htmlを取得
search_results_html = browser.page_source
print(search_results_html)

<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="ja" lang="ja" dir="ltr" id="R1"><head><style media="screen"></style>
<meta http-equiv="Content-Language" content="ja">
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<meta http-equiv="Content-Style-Type" content="text/css; charset=UTF-8">
<meta http-equiv="Content-Script-Type" content="text/javascript; charset=UTF-8">

<meta http-equiv="X-UA-Compatible" content="IE=Edge">
<meta name="mixi-check-robots" content="nodescription, noimage">
<meta name="thumbnail" content="https://assets.nikkei.jp/release/v3.2.193/parts/ds/images/common/icon_ogpnikkei.png">
<meta property="og:image" content="https://assets.nikkei.jp/release/v3.2.193/parts/ds/images/common/icon_ogpnikkei.png">
<meta property="fb:app_id" content="197388106979545">
<meta name="description" content="日経新聞電子版のプレスリリースページ。各企業・団体の新製品、新サービスのプレスリリース（報道機関向け発表資料）など最新情報をまとめて掲載します。個別の企業名や業種での検索も可能で、原則、発表当日に掲載。">
<meta name="news_keywords" content="プレスリリース,情報,通信,メディア,

In [6]:
# 記事本体の確認
url = "https://www.nikkei.com/article/DGXLRSP607403_W1A320C2000000/?au=0"
# URLを読み込む
browser.get(url)
# htmlを取得
search_results_html = browser.page_source
print(search_results_html)

<html lang="ja" data-service-name="kite-article"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1">
<!-- Block added by Adlock: start -->
<script type="text/javascript" src="https://nkispa.nikkei.com/fh/showad_d094700e_d.js" async=""></script><script type="text/javascript" src="//www.nikkei.com/adlock/injections/script.js?t=357067688&amp;p=6145852"></script><style type="text/css">.web-notification, [data-target="#newsletter-modal"], .wsapp, a[href^="https://twitter.com/home/?status="], a[href^="http://twitter.com/?status="], a[href^="http://www.facebook.com/share.php"], a[href^="http://www.reddit.com/submit"], a[href^="http://technorati.com/faves?add="], a[href^="http://www.facebook.com/sharer/sharer.php"], a[href^="https://pinterest.com/pin/create/"], a[href^="http://pinterest.com/pin/create/button/"], a[href^="https://form.typeform.com/"], a[href^="https://www.facebook.com/sharer/sharer.php"], div[class^="app_gdpr"], di

In [7]:
# プレスリリースのスクレイピング
import sys
import urllib.parse
import re
from html import unescape
import datetime
import os

from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary
from bs4 import BeautifulSoup

def get_article_to_text(search_results, file):

  for article_local_url in re.findall(r'<a href="(.*?)"', search_results, re.DOTALL):
    article_url = 'https://www.nikkei.com'+ article_local_url
    print(article_url)
    # 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
    browser.implicitly_wait(3)
    # URLを読み込む
    browser.get(article_url)
    # htmlを取得
    article_html = browser.page_source
    #print(article_html)
    header = re.search(r'<h1 class="title_tp0qjrp">(.*?)</h1>',article_html,re.DOTALL).group(1)
    date = re.search(r'<time datetime=(.*?)000Z">(.*?)</time>',article_html,re.DOTALL).group(2)
    print(header,date)
    # 記事本体からテキストのみを取り出し、テキストファイルに書き出す 
    a = re.search(r'<p class="paragraph_p15tm1hb">発表日:(.*?)すべての記事が読み放題',article_html,re.DOTALL).group(1)
    article = re.search(r'<p class="paragraph_p15tm1hb">(.*?)アプリで開く', a, re.DOTALL).group(1)
    article = article.replace('</p><p class="paragraph_p15tm1hb">', '\n').replace('</p></section><div class="div_d1961zuf"><a class="buttonStyle_bu8516w    styles_s1260nz6" href="','')
    article = article.replace('" data-rn-track="open_in_app" data-rn-track-category="button"><span class="">','')
    try:
      file.write(re.sub('<a.*?</a>', '', article))
      file.write('\n')
    except:
      print(re.sub('<a.*?</a>', '', article))
  return

# 企業名　と　発表年　で日経のプレスリリースを検索するURLを作成
# 検索キーワードをURLエンコードする
keywords = urllib.parse.quote('パナソニック AND 発表日：2021')
url = 'https://www.nikkei.com/pressrelease/?searchKeyword='+ keywords + '&au=0'

#　日経のプレスリリースを検索
# ブラウザーを起動
options = Options()
# 必要ならproxyサーバを設定する
#PROXY = '127.0.0.1:8080'
#options.add_argument('--proxy-server=http://%s' % PROXY)
# ブラウザはヘッドレス（表示しない）とする
options.add_argument('--headless')
# Chromeのドライバを得る
browser = webdriver.Chrome(options=options)

# 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
browser.implicitly_wait(3)
# URLを読み込む
browser.get(url)
# htmlを取得
search_results_html = browser.page_source
# 検索結果の件数からページ数を得る（検索結果は１ページあたり３０件までのため）
results_num = re.search(r'<div class="m-pageNation">.*?<p>(.*?)件中.*?</p>',search_results_html,re.DOTALL).group(1)

pages_num = -(-(int(results_num))//30)
#pages_num = 1  # 検索結果のはじめの１ページ（３０件）分のみに制限

file = open(urllib.parse.unquote(keywords) + 'articles.txt', 'a', encoding='UTF-8')

search_results = re.search(r'<h2 class="m-headline_text">(.*?)<p>',search_results_html,re.DOTALL).group(1)

get_article_to_text(search_results, file)

for i in range(pages_num-1):
  url = 'https://www.nikkei.com/pressrelease/?searchKeyword='+ keywords + '&hm='+ str(i+2)
  # 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
  browser.implicitly_wait(3)
  # URLを読み込む
  browser.get(url)
  # htmlを取得
  search_results_html = browser.page_source

  search_results = re.search(r'<h2 class="m-headline_text">(.*?)<p>',search_results_html,re.DOTALL).group(1)
    
  get_article_to_text(search_results, file)

# ブラウザを終了
browser.quit()
file.close()

https://www.nikkei.com/article/DGXLRSP624383_S1A221C2000000/?au=0
パナソニック、CATVデジタルセットトップボックス「TZ-BT9000BW」を発売 2021年12月22日 10:49
https://www.nikkei.com/article/DGXLRSP624338_R21C21A2000000/?au=0
JEITA、11月のパーソナルコンピュータ国内出荷実績を発表 2021年12月21日 15:22
https://www.nikkei.com/article/DGXLRSP624050_X11C21A2000000/?au=0
パナソニック、家電と食のサブスクリプションサービス「foodable（フ―ダブル）」に新コースを追加 2021年12月17日 11:36
https://www.nikkei.com/article/DGXLRSP623941_W1A211C2000000/?au=0
パナソニック、ネットワーク統合管理ソフトウェア「PPS」及びスイッチングハブ「MSシリーズ」などの機能を強化し提供開始 2021年12月16日 12:11
https://www.nikkei.com/article/DGXLRSP623879_V11C21A2000000/?au=0
シスコ、パナソニックと共同で「5Gショーケース」に放送事業者向け実証実験環境を追加 2021年12月15日 15:50
https://www.nikkei.com/article/DGXLRSP623875_V11C21A2000000/?au=0
パナソニック、ブルーレイディスクレコーダー ディーガ「DMR-ZR1」を発売 2021年12月15日 15:37
https://www.nikkei.com/article/DGXLRSP623637_T11C21A2000000/?au=0
パナソニックホームズ、全館空調システムを標準装備しZEHにも対応できる住まい「FORTINA X」を発売 2021年12月13日 14:59
https://www.nikkei.com/article/DGXLRSP623371_Z01C21A2000000/?au=0
三菱地所とパナソニックなど、「（仮称）泉パークタウン第6

パナソニックエコシステムズ、ベトナムで換気などIAQ（室内空気質）機器の新工場を稼働 2021年9月27日 11:55
https://www.nikkei.com/article/DGXLRSP618359_U1A920C2000000/
国内自動車メーカー5社と部品メーカー5社、「MBD（モデルベース開発）推進センター」に運営会員として参画 2021年9月24日 12:29
https://www.nikkei.com/article/DGXLRSP618338_U1A920C2000000/
ポラリスとパナソニック、IoT・AIを活用した要介護高齢者向け「短期滞在型自立支援サービス」を提供開始 2021年9月24日 11:03
https://www.nikkei.com/article/DGXLRSP618209_R20C21A9000000/
パナソニック、43V型液晶テレビ「4Kビエラ『レイアウトフリーテレビ』」を発売 2021年9月21日 15:21
https://www.nikkei.com/article/DGXLRSP618203_R20C21A9000000/
パナソニック、ルームエアコン「エオリア」LXシリーズを発売 2021年9月21日 15:21
https://www.nikkei.com/article/DGXLRSP618208_R20C21A9000000/
JEITA、8月のパーソナルコンピュータ国内出荷実績を発表 2021年9月21日 15:20
https://www.nikkei.com/article/DGXLRSP618146_X10C21A9000000/
パナソニック、サプライチェーン・ソフトウェアの専門企業であるBlue Yonderの買収を完了 2021年9月17日 17:10
https://www.nikkei.com/article/DGXLRSP618099_X10C21A9000000/
パナソニックアーキスケルトンデザイン、テクノストラクチャー工法2階建て住宅「BASIE（ベイシー）」を発売 2021年9月17日 12:01
https://www.nikkei.com/article/DGXLRSP618055_W1A910C2000000/
パナソニック、特別仕様のダイレクトドライブターンテー

スマートドライブとパナソニック、ETC2.0システムを活用した運行管理サービス実現のための共同実証を開始 2021年6月29日 14:21
https://www.nikkei.com/article/DGXLRSP613441_Z20C21A6000000/
パナソニック、サイバーエージェント番組制作拠点の8つのスタジオで「IT/IPプラットフォーム"KAIROS"」を受注 2021年6月29日 11:23
https://www.nikkei.com/article/DGXLRSP613324_V20C21A6000000/
パナソニック、ローカル5G含むマルチアクセス制御技術などの実証や検証を行えるラボを開設 2021年6月25日 16:20
https://www.nikkei.com/article/DGXLRSP613297_V20C21A6000000/
パナソニック、電動工具新ブランド「EXENA」2シリーズを発売 2021年6月25日 14:21
https://www.nikkei.com/article/DGXLRSP613192_U1A620C2000000/
パナソニック、女子プロテニスプレイヤー大坂なおみ選手とブランドアンバサダー契約の締結を発表 2021年6月24日 15:00
https://www.nikkei.com/article/DGXLRSP613183_U1A620C2000000/
パナソニック、「3D匠ねり」と「Wセンシング発酵」を搭載したホームベーカリー「ビストロ」SD-MDX4 他2機種を発売 2021年6月24日 14:23
https://www.nikkei.com/article/DGXLRSP613092_T20C21A6000000/
JEITA、5月のパーソナルコンピュータ国内出荷実績を発表 2021年6月23日 15:21
https://www.nikkei.com/article/DGXLRSP613082_T20C21A6000000/
パナソニック、ビルテナント向け高セキュリティ通信サービスとビル運営管理システムのサービス実証を開始 2021年6月23日 14:27
https://www.nikkei.com/article/DGXLRSP613075_T20C21A6000000/
パナ

パナソニック、エクセルシャノンと断熱性能の樹脂サッシを共同開発し真空断熱ガラスを国内の建築物用の樹脂サッシに導入 2021年4月14日 15:19
https://www.nikkei.com/article/DGXLRSP608563_U1A410C2000000/
パナソニックホームズ、CRE有効活用事業トータル提案事例のコンドミニアムタイプ宿泊施設「ホテルピアチェーレなんば」が開業 2021年4月14日 11:53
https://www.nikkei.com/article/DGXLRSP608547_U1A410C2000000/
パナソニック、頑丈ノートパソコン「TOUGHBOOK」のハイエンドモデルCF-33シリーズを国内法人向けに発売 2021年4月14日 10:53
https://www.nikkei.com/article/DGXLRSP608483_T10C21A4000000/
パナソニック、「液体洗剤・柔軟剤 自動投入」機能を搭載した大容量全自動洗濯機2機種を発売 2021年4月13日 15:01
https://www.nikkei.com/article/DGXLRSP608437_S1A410C2000000/
セイコーエプソンとエプソン販売、サテライトオフィス「阪急阪神ONS千里中央」で顔認証機能付き印刷システムを導入 2021年4月12日 17:50
https://www.nikkei.com/article/DGXLRSP608413_S1A410C2000000/
パナソニックグループ、フルモデルチェンジしたスタックオンタイプの製氷機「Aシリーズ」を発売 2021年4月12日 16:21
https://www.nikkei.com/article/DGXLRSP608131_X00C21A4000000/
パナソニック、「IoT対応スチーム&amp;可変圧力IHジャー炊飯器『おどり炊き』」SR-VSX1シリーズを発売 2021年4月7日 14:01
https://www.nikkei.com/article/DGXLRSP608030_W1A400C2000000/
パナソニック、クラウド管理型無線LANシステム「AIRRECT（エアレクト）」の発売を延期 2021年4月6日 14:18
https://www.nik

パナソニック、マイナス70℃を最長18日間保持できる真空断熱保冷ボックス「VIXELL（ビクセル）」を開発 2021年1月21日 12:15
https://www.nikkei.com/article/DGXLRSP603363_Z10C21A1000000/
パナソニック、大画面HUDが日産自の新型クロスオーバーSUV「Rogue」に採用 2021年1月19日 14:01
https://www.nikkei.com/article/DGXLRSP603304_Y1A110C2000000/
パナソニック、Sub6帯域を活用したローカル5Gスタンドアローンシステムの実験試験局免許を取得 2021年1月18日 16:31
https://www.nikkei.com/article/DGXLRSP603283_Y1A110C2000000/
パナソニック、AIカメラを活用した画像解析体験ができる「Vieurekaカメラ スターターキット」を発売 2021年1月18日 15:01
https://www.nikkei.com/article/DGXLRSP603262_Y1A110C2000000/
パナソニック、ラムサ・東京都市大と劇場・観覧施設の最適な計画・運用を支援する設計評価手法を確立 2021年1月18日 13:31
https://www.nikkei.com/article/DGXLRSP603202_V10C21A1000000/
パナソニック、壁掛形非常用放送設備 WK-EK300シリーズを発売 2021年1月15日 13:55
https://www.nikkei.com/article/DGXLRSP603168_U1A110C2000000/
メディクルード・EDGEMATRIX・NTTドコモなど、介護AIソリューション導入に向けた検証環境の構築・実証実験を開始 2021年1月14日 17:59
https://www.nikkei.com/article/DGXLRSP603132_U1A110C2000000/
パナソニック、Grabのプレミアム車両で「ナノイー X」による快適な移動空間を提供 2021年1月14日 14:35
https://www.nikkei.com/article/DGXLRSP603071_T10C21A1000